# Test Driven Development of Character class layout

This notebook serves two purposes:
- To develop the program (layout of classes, data structures, etc) for the interactive character sheet
- To provide an example of test driven development

## Objective

We want to be able to initialise/create a Character object that can return all of the essential information about about the D&D Character. It should also be able to level up and contain methods for all actions in combat.

## List of Requirements (Update this time)
1. Initialise a `Character` object
2. Initialiase `Character` with some details
    - A string for each of: `name`, `race`, and `character_class`
    - An integer for `level`, which defaults to 1
3. Make it that `Character` "has a" `Race` and "has a" `CharacterClass`
    - Each is a class of it's own
    - Build in the ability to check the string with a `__str__` method
4. Further develop the `Race` class
    - It has subclasses for each race from D&D 5e
    - The `__str__` method returns the race name
5. Further develop the `CharacterClass` class
    - It has subclasses for each character class from D&D 5e
    - The `__str__` method returns the character class name
6. Initialise a `Player` class which "is a" `Character`
7. Initialise a `NonPlayer` class which "is a" `Character`
8. Initialise a `Weapon` class
9. Initialise a `Spell` class
10. Initialise an `Ability` class
    - With each ability as a subclass (`Strength` etc...)
11. Each `Ability` subclass has skills including a saving throw
    - Skills and saving throws are kept in a dictionary called `proficiencies`
    - `proficiencies` key:value pairs are all `skill : <bool>`
12. Add to `Ability` class
    - `Ability` has the `check()` method and `modifier` and `score` properties
13. 

#### First, import the testing modules:

In [1]:
# Set the file name for unit testing iwth ipytest
__file__ = 'character_scripting.ipynb'

import pytest
import ipytest.magics

## 1. Initialise a `Character` object

#### Make sure the tests are defined *before* the main code is written:

In [2]:
%%run_pytest -v --tb=line

def test_Character_can_be_created():
    assert Character()

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 1 item

character_scripting.py::test_Character_can_be_created <- <ipython-input-2-24813ad9a017> FAILED [100%]

================================== FAILURES ===================================
<ipython-input-2-24813ad9a017>:3: NameError: name 'Character' is not defined
========================== 1 failed in 0.26 seconds ===========================


#### Test failed (because we are yet to define `Character`). "Refactor" the code:

In [3]:
%%run_pytest

class Character(object):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 1 item

character_scripting.py .                                                 [100%]

========================== 1 passed in 0.07 seconds ===========================


### [This process](https://en.wikipedia.org/wiki/Test-driven_development#Test-driven_development_cycle) should be repeated for each new requirement
1. **Write** the test(s)
```python
def test_<test_description>():
    # Some code (potentially)
    assert true_statement
```
2. **Run** the tests
```python
%%run_pytest -v --tb=line
```
3. **Refactor** (update or write new code - including tests) until all tests pass
4. **Repeat** for the next requirement

## 2. Initialiase `Character` with some details
- A string for each of: `name`, `race`, and `character_class`
- An integer for `level`, which defaults to 1

1. **Write** the test(s)

In [4]:
%%run_pytest -v --tb=line

def test_Character_for_name():
    assert "Merret" == Character("Merret","Halfling","Ranger", 8).name
def test_Character_for_race():
    assert "Halfling" == Character("Merret","Halfling","Ranger", 8)._race
def test_Character_for_character_class():
    assert "Ranger" == Character("Merret","Halfling","Ranger", 8).character_class
def test_Character_for_level():
    assert 8 == Character("Merret","Halfling","Ranger", 8).level
def test_Character_for_level_default():
    assert 1 == Character("Merret","Halfling","Ranger").level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 6 items

character_scripting.py::test_Character_can_be_created <- <ipython-input-2-24813ad9a017> PASSED [ 16%]
character_scripting.py::test_Character_for_name <- <ipython-input-4-eab3a64ac030> FAILED [ 33%]
character_scripting.py::test_Character_for_race <- <ipython-input-4-eab3a64ac030> FAILED [ 50%]
character_scripting.py::test_Character_for_character_class <- <ipython-input-4-eab3a64ac030> FAILED [ 66%]
character_scripting.py::test_Character_for_level <- <ipython-input-4-eab3a64ac030> FAILED [ 83%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-4-eab3a64ac030> FAILED [100%]

================================== FAILURES ======================

2. **Run** the tests

In [5]:
%%run_pytest -v --tb=line

class Character(object):
    
    def __init__(self, name, race, character_class, level=1):
        self.name = name
        self._race = race
        self.character_class = character_class
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 6 items

character_scripting.py::test_Character_can_be_created <- <ipython-input-2-24813ad9a017> FAILED [ 16%]
character_scripting.py::test_Character_for_name <- <ipython-input-4-eab3a64ac030> PASSED [ 33%]
character_scripting.py::test_Character_for_race <- <ipython-input-4-eab3a64ac030> PASSED [ 50%]
character_scripting.py::test_Character_for_character_class <- <ipython-input-4-eab3a64ac030> PASSED [ 66%]
character_scripting.py::test_Character_for_level <- <ipython-input-4-eab3a64ac030> PASSED [ 83%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-4-eab3a64ac030> PASSED [100%]

================================== FAILURES ======================

3. **Refactor** - The initial test does not include any inputs to `Character`. Include default values.

In [6]:
%%run_pytest -v --tb=line

class Character(object):
    
    def __init__(self, name="Merret", race="Halfling", character_class="Ranger", level=1):
        self.name = name
        self._race = race
        self.character_class = character_class
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 6 items

character_scripting.py::test_Character_can_be_created <- <ipython-input-2-24813ad9a017> PASSED [ 16%]
character_scripting.py::test_Character_for_name <- <ipython-input-4-eab3a64ac030> PASSED [ 33%]
character_scripting.py::test_Character_for_race <- <ipython-input-4-eab3a64ac030> PASSED [ 50%]
character_scripting.py::test_Character_for_character_class <- <ipython-input-4-eab3a64ac030> PASSED [ 66%]
character_scripting.py::test_Character_for_level <- <ipython-input-4-eab3a64ac030> PASSED [ 83%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-4-eab3a64ac030> PASSED [100%]

========================== 6 passed in 0.22 seconds ==============

## 3. Make it that `Character` "has a" `Race` and "has a" `CharacterClass`
- Each is a class of it's own
- Build in the ability to check the string with a `__str__` method

In [7]:
def test_Character_has_a_Race():
    assert isinstance(Character()._race, Race)
def test_Character_has_a_CharacterClass():
    assert isinstance(Character().character_class, CharacterClass)


In [8]:
%%run_pytest -v --tb=line

class Race(object):
    
    def __init__(self, race):
        self._race = race
    
    def __str__(self):
        return self._race


class CharacterClass(object):
    
    def __init__(self, character_class):
        self.character_class = character_class
    
    def __str__(self):
        return self.character_class


class Character(object):
    
    def __init__(self, name="Merret", race="Halfling", character_class="Ranger", level=1):
        self.name = name
        self._race = Race(race) # Changed Line
        self.character_class = CharacterClass(character_class) # Changed Line
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 8 items

character_scripting.py::test_Character_can_be_created <- <ipython-input-2-24813ad9a017> PASSED [ 12%]
character_scripting.py::test_Character_for_name <- <ipython-input-4-eab3a64ac030> PASSED [ 25%]
character_scripting.py::test_Character_for_race <- <ipython-input-4-eab3a64ac030> FAILED [ 37%]
character_scripting.py::test_Character_for_character_class <- <ipython-input-4-eab3a64ac030> FAILED [ 50%]
character_scripting.py::test_Character_for_level <- <ipython-input-4-eab3a64ac030> PASSED [ 62%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-4-eab3a64ac030> PASSED [ 75%]
character_scripting.py::test_Character_has_a_Race <- <ipython-input

#### The new requirement breaks the `test_Character_for_race` and the `test_Character_for_character_class` tests. Refactor the tests to make use of the `__str__` method

In [9]:
%%run_pytest

def test_Character_for_race():
    assert 'Halfling' == str(Character(race = "Halfling")._race)
def test_Character_for_character_class():
    assert 'Ranger' == str(Character(character_class = "Ranger").character_class)

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 8 items

character_scripting.py ........                                          [100%]

========================== 8 passed in 0.11 seconds ===========================


## 4. Further develop the `Race` class
   - It has subclasses for each race from D&D 5e
   - The `__str__` method returns the race name

In [10]:
def test_Dwarf_is_a_Race():
    assert isinstance(Dwarf(),Race)
def test_Elf_is_a_Race():
    assert isinstance(Elf(),Race)
def test_Halfling_is_a_Race():
    assert isinstance(Halfling(),Race)
def test_Human_is_a_Race():
    assert isinstance(Human(),Race)
def test_Dragonborn_is_a_Race():
    assert isinstance(Dragonborn(),Race)
def test_Gnome_is_a_Race():
    assert isinstance(Gnome(),Race)
def test_HalfElf_is_a_Race():
    assert isinstance(HalfElf(),Race)
def test_HalfOrc_is_a_Race():
    assert isinstance(HalfOrc(),Race)
def test_Tiefling_is_a_Race():
    assert isinstance(Tiefling(),Race)

In [11]:
%%run_pytest -v --tb=line

class Race(object):
    
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__ # return the name of the class

    
class Dwarf(Race):
    pass
class Elf(Race):
    pass
class Halfling(Race):
    pass
class Human(Race):
    pass
class Dragonborn(Race):
    pass
class Gnome(Race):
    pass
class HalfElf(Race):
    pass
class HalfOrc(Race):
    pass
class Tiefling(Race):
    pass


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 17 items

character_scripting.py::test_Dwarf_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [  5%]
character_scripting.py::test_Elf_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 11%]
character_scripting.py::test_Halfling_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 17%]
character_scripting.py::test_Human_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 23%]
character_scripting.py::test_Dragonborn_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 29%]
character_scripting.py::test_Gnome_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 35%]
character_scripting.py::test_HalfElf_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 41%]
cha

#### Must update assignment of `_race` in `Character`

In [12]:
%%run_pytest

class Character(object):
    
    def __init__(self, name="Merret", race="Halfling", character_class="Ranger", level=1):
        self.name = name
        self._race = globals()[race.title()]() # Changed Line
        self.character_class = CharacterClass(character_class)
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 17 items

character_scripting.py .................                                 [100%]

========================== 17 passed in 0.18 seconds ==========================


## 5. Further develop the `CharacterClass` class
   - It has subclasses for each character class from D&D 5e
   - The `__str__` method returns the character class name

In [13]:
def test_Barbarian_is_a_CharacterClass():
    assert isinstance(Barbarian(),CharacterClass)
def test_Bard_is_a_CharacterClass():
    assert isinstance(Bard(),CharacterClass)
def test_Cleric_is_a_CharacterClass():
    assert isinstance(Cleric(),CharacterClass)
def test_Druid_is_a_CharacterClass():
    assert isinstance(Druid(),CharacterClass)
def test_Fighter_is_a_CharacterClass():
    assert isinstance(Fighter(),CharacterClass)
def test_Monk_is_a_CharacterClass():
    assert isinstance(Monk(),CharacterClass)
def test_Paladin_is_a_CharacterClass():
    assert isinstance(Paladin(),CharacterClass)
def test_Ranger_is_a_CharacterClass():
    assert isinstance(Ranger(),CharacterClass)
def test_Rogue_is_a_CharacterClass():
    assert isinstance(Rogue(),CharacterClass)
def test_Sorcerer_is_a_CharacterClass():
    assert isinstance(Sorcerer(),CharacterClass)
def test_Warlock_is_a_CharacterClass():
    assert isinstance(Warlock(),CharacterClass)
def test_Wizard_is_a_CharacterClass():
    assert isinstance(Wizard(),CharacterClass)

In [14]:
%%run_pytest

class CharacterClass(object):
    
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__ # return the name of the class

    
class Barbarian(CharacterClass):
    pass
class Bard(CharacterClass):
    pass
class Cleric(CharacterClass):
    pass
class Druid(CharacterClass):
    pass
class Fighter(CharacterClass):
    pass
class Monk(CharacterClass):
    pass
class Paladin(CharacterClass):
    pass
class Ranger(CharacterClass):
    pass
class Rogue(CharacterClass):
    pass
class Sorcerer(CharacterClass):
    pass
class Warlock(CharacterClass):
    pass
class Wizard(CharacterClass):
    pass

# Same problem will occur as when changing Race, so adjust this for CharacterClass now
class Character(object):
    
    def __init__(self, name="Merret", race="Halfling", character_class="Ranger", level=1):
        self.name = name
        self._race = globals()[race.title()]()
        self.character_class = globals()[character_class.title()]() # Changed Line
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 29 items

character_scripting.py .............................                     [100%]

========================== 29 passed in 0.23 seconds ==========================


## 6. Initialise a `Player` class which "is a" `Character`

In [15]:
def test_Player_is_a_Character():
    assert isinstance(Player(),Character)

In [16]:
%%run_pytest

class Player(Character):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 30 items

character_scripting.py ..............................                    [100%]

========================== 30 passed in 0.24 seconds ==========================


## 7. Initialise a `NonPlayer` class which "is a" `Character`

In [17]:
def test_NonPlayer_is_a_Character():
    assert isinstance(NonPlayer(),Character)

In [18]:
%%run_pytest

class NonPlayer(Character):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 31 items

character_scripting.py ...............................                   [100%]

========================== 31 passed in 0.25 seconds ==========================


## 8. Initialise a `Weapon` class

In [19]:
def test_Weapon_can_be_created():
    assert Weapon()

In [20]:
%%run_pytest

class Weapon(object):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 32 items

character_scripting.py ................................                  [100%]

========================== 32 passed in 0.26 seconds ==========================


## 9. Initialise a `Spell` class

In [21]:
def test_Spell_can_be_created():
    assert Spell()

In [22]:
%%run_pytest

class Spell(object):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 33 items

character_scripting.py .................................                 [100%]

========================== 33 passed in 0.29 seconds ==========================


## 10. Initialise an `Ability` class

In [23]:
def test_Ability_can_be_created():
    assert Ability()

In [24]:
%%run_pytest

class Ability(object):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 34 items

character_scripting.py ..................................                [100%]

========================== 34 passed in 0.28 seconds ==========================


## 11. Add to `Ability` class
   - `Ability` has the `check()` method and `modifier` and `score` properties

In [25]:
def test_Ability_has_check():
    assert Ability().check()
def test_Ability_has_modifier():
    assert Ability().modifier is not None
def test_Ability_has_score():
    assert Ability().score

In [26]:
%%run_pytest -v --tb=line

import random

class Ability(object):
    
    def __init__(self, score=10):
        self.score = score
        self.modifier = int((score - 10) / 2)
        
    def check(self):
        return random.randint(1,20) + self.modifier
    

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 37 items

character_scripting.py::test_Dwarf_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [  2%]
character_scripting.py::test_Elf_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [  5%]
character_scripting.py::test_Halfling_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [  8%]
character_scripting.py::test_Human_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 10%]
character_scripting.py::test_Dragonborn_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 13%]
character_scripting.py::test_Gnome_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 16%]
character_scripting.py::test_HalfElf_is_a_Race <- <ipython-input-10-1b7e05b37631> PASSED [ 18%]
cha

## 12. Each ability (`Strength` etc...) is a subclass of `Ability`

In [27]:
def test_Strength_is_an_Ability():
    assert isinstance(Strength(),Ability)
def test_Dexterity_is_an_Ability():
    assert isinstance(Dexterity(),Ability)
def test_Constitution_is_an_Ability():
    assert isinstance(Constitution(),Ability)
def test_Intelligence_is_an_Ability():
    assert isinstance(Intelligence(),Ability)
def test_Wisdom_is_an_Ability():
    assert isinstance(Wisdom(),Ability)
def test_Charisma_is_an_Ability():
    assert isinstance(Charisma(),Ability)

In [28]:
%%run_pytest

class Strength(Ability):
    pass
class Dexterity(Ability):
    pass
class Constitution(Ability):
    pass
class Intelligence(Ability):
    pass
class Wisdom(Ability):
    pass
class Charisma(Ability):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 43 items

character_scripting.py ...........................................       [100%]

========================== 43 passed in 0.31 seconds ==========================


## 13. Each `Ability` subclass has skills
   - Skills and saving throws are kept in a dictionary called `proficiencies`
   - `proficiencies` key:value pairs are all `skill : <bool>`

In [29]:
# Has skills
def test_Strength_has_skills():
    assert isinstance(Strength().proficiencies,dict)
def test_Dexterity_has_skills():
    assert isinstance(Dexterity().proficiencies,dict)
def test_Constitution_has_skills():
    assert isinstance(Constitution().proficiencies,dict)
def test_Intelligence_has_skills():
    assert isinstance(Intelligence().proficiencies,dict)
def test_Wisdom_has_skills():
    assert isinstance(Wisdom().proficiencies,dict)
def test_Charisma_has_skills():
    assert isinstance(Charisma().proficiencies,dict)

# Are booleans
def test_Strength_skills_are_bool():
    assert isinstance(list(Strength().proficiencies.values())[0],bool)
def test_Dexterity_skills_are_bool():
    assert isinstance(list(Dexterity().proficiencies.values())[0],bool)
def test_Constitution_skills_are_bool():
    assert isinstance(list(Constitution().proficiencies.values())[0],bool)
def test_Intelligence_skills_are_bool():
    assert isinstance(list(Intelligence().proficiencies.values())[0],bool)
def test_Wisdom_has_skills_are_bool():
    assert isinstance(list(Wisdom().proficiencies.values())[0],bool)
def test_Charisma_has_skills_are_bool():
    assert isinstance(list(Charisma().proficiencies.values())[0],bool)

In [30]:
%%run_pytest

#################### DOUBLE CHECK i HAVE THE CORRECT LIST OF SKILLS

class Strength(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throw", "Athletics"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Dexterity(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throw", "Acrobatics", "Sleight of Hand",
                     "Stealth"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Constitution(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throw"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Intelligence(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throw", "History", "Arcana", "Religion"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Wisdom(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throw", "Medicine", "Perception", "Insight"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Charisma(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throw", "Persuasion", "Deception",
                      "Intimidation", "Performance"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 55 items

character_scripting.py ................................................. [ 89%]
......                                                                   [100%]

========================== 55 passed in 0.39 seconds ==========================


In [31]:
# Main superclass for the Class of a Character
# Includes all aspects that are common to all classes
class CharacterClass(object):
    
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    

In [32]:
# Example of specific Character Classes - "is a" CharacterClass
class Barbarian(CharacterClass):
    pass

class Bard(CharacterClass):
    pass

class Ranger(CharacterClass):
    pass

class Sorceror(CharacterClass):
    pass


In [33]:
# Main superclass for the Race of a Character
class Race(object):
    
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    

In [34]:
# Example of a specific Character Race - "is a" Race
class Halfling(Race):
    pass

class Tiefling(Race):
    pass


In [35]:
# Main class for all Characters - "has a" Race and Class
class Character(object):
    
    def __init__(self, name, race, character_class, level=1):
        self.name = name
        self._class = globals()[character_class.title()]()
        self._race = globals()[race.title()]()
        self.level = level
        

In [36]:
# Main class for Player Characters - "is a" Character
class Player(Character):
    
    def __init__(self, name, race, character_class, ability_scores, level=1):
        super().__init__(name, race, character_class, level)
        self.__spellcaster = str(self._class) in [
            "Bard", "Druid", "Ranger", "Sorceror", "Wizard", "Warlock"]
        self.ability_scores = {
            "Strength" : ability_scores[0],
            "Dexterity" : ability_scores[1],
            "Constitution" : ability_scores[2],
            "Intelligence" : ability_scores[3],
            "Wisdom" : ability_scores[4],
            "Charisma" : ability_scores[5]
        }
        
    def __str__(self):
        return "A level {0} {1} {2} called {3}".format(
            self.level, 
            str(self._race).title(),
            str(self._class),
            self.name.title())
    
    def attack(self):
        pass

In [37]:
# Main class for Non-Player Characters - "is a" Character
class NonPlayer(Character):
    pass

In [38]:
merret = Player("Merret Strongheart",
                   "Halfling",
                   "ranger",
                   [12, 20, 12, 8, 16, 8],
                   9)

In [39]:
print(merret._class)

Ranger


In [40]:
merret._Player__spellcaster

True

In [41]:
print(merret)

A level 9 Halfling Ranger called Merret Strongheart


In [42]:
despair = Player("Despair", "Tiefling", "Sorceror", [9, 11, 15, 14, 13, 20], 8)

In [43]:
despair._Player__spellcaster

True

In [44]:
print(despair)

A level 8 Tiefling Sorceror called Despair
